In [1]:
from gas import GASModel
from utils import generate_timeseries, generate_dataset

import torch.nn as nn
import torch.optim as optim

import plotly.graph_objects as go
import numpy as np

Generate time series and dataset

In [2]:
GROWING = True
t, y = generate_timeseries(GROWING)

In [3]:
# split the timeseries
MAX_INDEX_TRAIN = 1500
MAX_INDEX_TEST = 2500
y_train = y[:MAX_INDEX_TRAIN]
y_test = y[MAX_INDEX_TRAIN: MAX_INDEX_TEST]

In [4]:
# generate the dataset as tensor of shape 
# (n_timeseries, ts_length, n_features) for inputs
# (n_timeseries, el_to_predict) for labels
TS_LENGTH = 200
EL_TO_PREDICT = 50
y_train, lab_train = generate_dataset(y_train, TS_LENGTH, EL_TO_PREDICT)
y_test, lab_test = generate_dataset(y_test, TS_LENGTH, EL_TO_PREDICT)

Initialized the model

In [5]:
# initialize gas params
eta_mu = 0.999
eta_sigma2 = 0.999

# the encoder of the time series is just a flattener of the time dimension
ts_encoder = nn.Flatten()
# the output model is a feedforward network
# the output of ts_encoder is (batch, ts_length)
# the additional info is (batch, ts_length * 2 * n_features)
input_dim = TS_LENGTH + 2*TS_LENGTH   

HID_SIZE_1 = 100
HID_SIZE_2 = 100
output_model = nn.Sequential(nn.Linear(input_dim, HID_SIZE_1),
                                nn.ReLU(),
                                nn.Linear(HID_SIZE_1, HID_SIZE_2),
                                nn.ReLU(),
                                nn.Linear(HID_SIZE_2, EL_TO_PREDICT)
                                )

model = GASModel(ts_encoder, eta_mu, eta_sigma2, output_model)

Define train, evaluate and plot result functions

In [6]:
def train_model(model, criterion, optimizer, epochs, y_train, lab_train):

    for epoch in range(epochs):  # loop over the dataset multiple times

        running_loss = 0.0

        for inputs, labels in zip(y_train, lab_train):
            # the first dimension must be batch_size (i.e. 1)
            inputs = inputs.unsqueeze(0)
            labels = labels.unsqueeze(0)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs.float())
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
        print('[%d] loss: %.10f' %
                (epoch + 1, running_loss / y_train.shape[0]))

    print('Finished Training')

    return model


def eval_model(model, y_all):

    model.eval()
    y_pred = []
    for inputs in y_all:
        inputs = inputs.unsqueeze(0)    # first dimension must be batch_size
        outputs = model(inputs.float())
        outputs = outputs.squeeze()   # no need for batch_size here
        y_pred.append(outputs.detach().numpy())

    y_pred = np.array(y_pred)
    #y_pred = y_pred.reshape(-1)

    return y_pred


def plot_results(t, y, y_pred, max_index_test, max_index_train, ts_length):
    #plot with plotly with a line where the training set ends
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=t[0:max_index_test], y=y, mode='lines', name='Actual'))
    fig.add_trace(go.Scatter(x=t[ts_length:max_index_test], y=y_pred, mode='lines', name='Predicted'))
    fig.add_trace(go.Scatter(x=[t[max_index_train], t[max_index_train]], y=[-20, 150], mode='lines', name='Training Set End'))
    fig.update_layout(title='Dampened Sinusoid', xaxis_title='Time (s)', yaxis_title='Amplitude')
    fig.show()

Run the experiment

In [7]:
# train the model
criterion = nn.MSELoss()
LR = 1e-4
optimizer = optim.Adam(model.parameters(), lr=LR)
epochs = 400

model = train_model(model, criterion, optimizer, epochs, y_train, lab_train)

[1] loss: 2322.1878802960
[2] loss: 1797.8569523738
[3] loss: 1018.6064652663
[4] loss: 403.7742209801
[5] loss: 140.2757700407
[6] loss: 63.4356864049
[7] loss: 46.0164321753
[8] loss: 41.7767686844
[9] loss: 40.2301026583
[10] loss: 39.1814789038
[11] loss: 38.2115468979
[12] loss: 37.2565263235
[13] loss: 36.3093531315
[14] loss: 35.3707439716
[15] loss: 34.4423097739
[16] loss: 33.5249778766
[17] loss: 32.6197721408
[18] loss: 31.7275895889
[19] loss: 30.8488712402
[20] loss: 29.9842875004
[21] loss: 29.1343772228
[22] loss: 28.2994384857
[23] loss: 27.4799402494
[24] loss: 26.6762501460
[25] loss: 25.8886525723
[26] loss: 25.1174887052
[27] loss: 24.3630543489
[28] loss: 23.6256240698
[29] loss: 22.9054768452
[30] loss: 22.2028653347
[31] loss: 21.5178274742
[32] loss: 20.8507036246
[33] loss: 20.2019620492
[34] loss: 19.5716455258
[35] loss: 18.9608645714
[36] loss: 18.3669668344
[37] loss: 17.7934190677
[38] loss: 17.2381281027
[39] loss: 16.7014044432
[40] loss: 16.1840421053
[

In [8]:
# evaluate the model on the entire dataset
y_all, lab_all = generate_dataset(y, TS_LENGTH, EL_TO_PREDICT)

y_pred = eval_model(model, y_all)
y_all.shape, lab_all.shape, y_pred.shape

(torch.Size([46, 200, 1]), torch.Size([46, 50]), (46, 50))

In [9]:
y_pred = y_pred.reshape(-1)
plot_results(t, y, y_pred, MAX_INDEX_TEST, MAX_INDEX_TRAIN, TS_LENGTH)

In [10]:
#Compute the total error on the test set.
error = 0
for i in range(MAX_INDEX_TRAIN, MAX_INDEX_TEST):
    error = error + (y[i] - y_pred[i-TS_LENGTH])**2
error = error/(MAX_INDEX_TEST - MAX_INDEX_TRAIN)
print(error)

434.03922102573944
